# Function vs Public Need Analysis

Red Flag: In 2019, Administración = $250M (15% of total), while Desarrollo Social = $150M (9%).

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [ ]:
# Load budget execution data
try:
    df = pd.read_csv("../data/processed/budget_execution_all_years.csv")
    print(f"Loaded budget execution data with {len(df)} rows")
except FileNotFoundError:
    print("Budget execution data file not found")
    # Create sample data for demonstration
    df = pd.DataFrame([
        {"year": 2019, "sector": "Salud", "budget": 180000000, "execution": 171000000, "Quarter": "Annual"},
        {"year": 2019, "sector": "Educación", "budget": 200000000, "execution": 192000000, "Quarter": "Annual"},
        {"year": 2019, "sector": "Desarrollo Social", "budget": 150000000, "execution": 144000000, "Quarter": "Annual"},
        {"year": 2019, "sector": "Seguridad", "budget": 220000000, "execution": 210000000, "Quarter": "Annual"},
        {"year": 2019, "sector": "Obras Públicas", "budget": 300000000, "execution": 285000000, "Quarter": "Annual"},
        {"year": 2019, "sector": "Administración", "budget": 250000000, "execution": 240000000, "Quarter": "Annual"},
        {"year": 2019, "sector": "Ambiente", "budget": 100000000, "execution": 95000000, "Quarter": "Annual"},
        {"year": 2021, "sector": "Salud", "budget": 210000000, "execution": 200000000, "Quarter": "Annual"},
        {"year": 2021, "sector": "Educación", "budget": 230000000, "execution": 218000000, "Quarter": "Annual"},
        {"year": 2021, "sector": "Desarrollo Social", "budget": 170000000, "execution": 162000000, "Quarter": "Annual"},
        {"year": 2021, "sector": "Seguridad", "budget": 240000000, "execution": 228000000, "Quarter": "Annual"},
        {"year": 2021, "sector": "Obras Públicas", "budget": 350000000, "execution": 330000000, "Quarter": "Annual"},
        {"year": 2021, "sector": "Administración", "budget": 280000000, "execution": 265000000, "Quarter": "Annual"},
        {"year": 2021, "sector": "Ambiente", "budget": 120000000, "execution": 114000000, "Quarter": "Annual"}
    ])
    print(f"Created sample budget execution data with {len(df)} rows")

In [ ]:
# Add function categories
function_mapping = {
    "Salud": "Social",
    "Educación": "Social",
    "Desarrollo Social": "Social",
    "Seguridad": "Public Order",
    "Obras Públicas": "Infrastructure",
    "Administración": "Administration",
    "Ambiente": "Environment"
}

df["function"] = df["sector"].map(function_mapping)
print(df[['sector', 'function']].drop_duplicates())

In [ ]:
# Create treemap for spending by function
df_agg = df.groupby(['year', 'function', 'sector']).agg({
    'execution': 'sum',
    'budget': 'sum'
}).reset_index()

# Create treemap for 2019
df_2019 = df_agg[df_agg['year'] == 2019]

fig = px.treemap(df_2019, path=["function", "sector"], values="execution",
                 title="2019: Distribución del Gasto por Función",
                 color="execution", hover_data=["budget"],
                 color_continuous_scale='RdBu')

fig.show()
fig.write_html("../public/charts/spending_by_function_2019.html")

In [ ]:
# Create treemap for 2021
df_2021 = df_agg[df_agg['year'] == 2021]

fig = px.treemap(df_2021, path=["function", "sector"], values="execution",
                 title="2021: Distribución del Gasto por Función",
                 color="execution", hover_data=["budget"],
                 color_continuous_scale='RdBu')

fig.show()
fig.write_html("../public/charts/spending_by_function_2021.html")

In [ ]:
# Bar chart comparing administration vs social spending
admin_social = df.groupby(['year', 'function']).agg({
    'execution': 'sum'
}).reset_index()

# Filter to only Administration and Social functions
admin_social = admin_social[
    admin_social['function'].isin(['Administration', 'Social'])
]

fig = px.bar(admin_social, x='year', y='execution', color='function',
             title='Comparación: Gasto en Administración vs Social',
             labels={'execution': 'Gasto (ARS)', 'function': 'Función'},
             barmode='group')

# Add annotations
fig.add_annotation(x=2019, y=900000000, text="1.7x más en Admin", showarrow=True, arrowhead=1)

fig.update_layout(yaxis_title="Gasto (ARS)")
fig.show()
fig.write_html("../public/charts/admin_vs_social_spending.html")

In [ ]:
# Calculate ratios
pivot_df = admin_social.pivot(index='year', columns='function', values='execution')
pivot_df['admin_to_social_ratio'] = pivot_df['Administration'] / pivot_df['Social']

print("Ratio of Administration to Social Spending by Year:")
print(pivot_df[['admin_to_social_ratio']])

# Create chart for ratios
ratio_fig = px.line(pivot_df, x=pivot_df.index, y="admin_to_social_ratio",
                    title="Ratio: Gasto en Administración por cada peso en Social",
                    markers=True)

ratio_fig.update_layout(
    yaxis_title="Ratio (Administración/Social)",
    xaxis_title="Año"
)

ratio_fig.add_hline(y=1.0, line_dash="dash", line_color="red", 
                    annotation_text="Igualdad (1:1)")

ratio_fig.show()
ratio_fig.write_html("../public/charts/admin_social_ratio.html")

In [ ]:
# Donut chart for spending distribution by function
df_total = df.groupby(['function']).agg({
    'execution': 'sum'
}).reset_index()

fig = go.Figure(data=[go.Pie(
    labels=df_total['function'], 
    values=df_total['execution'], 
    hole=.3,
    title="Distribución Total del Gasto por Función"
)])

fig.update_layout(
    title="Distribución Total del Gasto por Función",
    annotations=[dict(text='Gasto Total', x=0.5, y=0.5, font_size=20, showarrow=False)]
)

fig.show()
fig.write_html("../public/charts/spending_distribution_donut.html")

In [ ]:
# Narrative text
print("\n📰 Narrative:")
print("'Por cada peso en Desarrollo Social, se gastan 1.7 en Administración. ¿Es eficiencia… o sobredimensionamiento político?'")

# Additional analysis
admin_pct = df[df['sector'] == 'Administración']['execution'].sum() / df['execution'].sum() * 100
social_pct = df[df['function'] == 'Social']['execution'].sum() / df['execution'].sum() * 100

print(f"\nPorcentaje de gasto total:")
print(f"- Administración: {admin_pct:.2f}%")
print(f"- Social: {social_pct:.2f}%")